# Final Project - Introduction to Data Science

**Team members:**

1. Nguyễn Hoàng Trung Kiên (Leader) - 22127478

2. Quách Trần Quán Vinh - 22127460

3. Ngô Hải Bằng - 22127460

# I. Preprocessing:

### Importing libraries

In [70]:
import pandas as pd
import re

## 1. Data cleaning

#### Read data ####

In [71]:
df = pd.read_csv('houses.csv')
df

,Price,Rooms,Type,Status,Parking Spaces,Land Size,Building Size,Architecture Style,County,Provider
0,"USD $2,200 per month","3 bedroom(s), 2 bathroom(s)",House,Active,NaN,NaN,136.20 m2,Other,Mohave,"Black Mountain Valley Realty, Inc"
1,"USD $3,000 per month","2 bedroom(s), 3 bathroom(s)",Apartment,Active,NaN,161.87 m2,205.13 m2,Other,PHILADELPHIA,Compass
2,"USD $1,195 per month","1 bedroom(s), 1 bathroom(s)",Apartment,Active,NaN,768.90 m2,NaN,Other,PHILADELPHIA,OCF Realty
3,"USD $8,000 per month","4 bedroom(s), 3 bathroom(s)",Townhouse,Active,NaN,728.43 m2,213.68 m2,Other,Ventura,eXp Realty of California Inc.
4,"USD $5,600 per month","4 bedroom(s), 3 bathroom(s), 1 basement(s)",House,Active,4.0,849.84 m2,266.72 m2,Colonial,FAIRFAX,Keller Williams Fairfax Gateway
...,...,...,...,...,...,...,...,...,...,...
3995,"USD $2,050 per month","3 bedroom(s), 2 bathroom(s)",Other,Active,NaN,566.56 m2,175.03 m2,Contemporary,Maricopa,Realty ONE Group
3996,"USD $1,300 per month","2 bedroom(s), 1 bathroom(s)",Other,Active,NaN,930.78 m2,104.98 m2,Ranch,Maricopa,Real Living SJ Fowler Real Estate
3997,"USD $3,499 per month","4 bedroom(s), 3 bathroom(s)",Other,Active,NaN,890.31 m2,345.79 m2,Contemporary,Maricopa,HomeSmart
3998,"USD $10,900 per month","5 bedroom(s), 7 bathroom(s)",Other,Active,NaN,"1,254.53 m2",653.94 m2,Other,Maricopa,Russ Lyon Sotheby's International Realty


#### Standardize data ####

Here we have columns ```Price``` , ```Building Size``` and ```Land Size``` need to be converted to float type

In [73]:
for i in range(len(df['Price'])):
    if df['Price'][i] != df['Price'][i]:
        df['Price'] = df['Price'].apply(lambda x: float(x.split('$')[1].split(' ')[0].replace(',', '')) if isinstance(x, str) else x)

In [75]:
for i in range(len(df['Building Size'])):
    if df['Building Size'][i] != df['Building Size'][i]:
        df['Building Size'] = df['Building Size'].apply(lambda x: float(x.split(' ')[0].replace(',', '')) if isinstance(x, str) else x)

In [76]:
for i in range(len(df['Land Size'])):
    if df['Land Size'][i] != df['Land Size'][i]:
        df['Land Size'] = df['Land Size'].apply(lambda x: float(x.split(' ')[0].replace(',', '')) if isinstance(x, str) else x)

```Rooms``` column will be converted into 3 other columns named ```Bedrooms```, ```Bathrooms``` and ```Basements```

In [74]:
bedrooms = df['Rooms'].apply(lambda x: int(re.search(r'(\d+) bedroom\(s\)', x).group(1)) if isinstance(x, str) and re.search(r'(\d+) bedroom\(s\)', x) else 0)
bathrooms = df['Rooms'].apply(lambda x: int(re.search(r'(\d+) bathroom\(s\)', x).group(1)) if isinstance(x, str) and re.search(r'(\d+) bathroom\(s\)', x) else 0)
basements = df['Rooms'].apply(lambda x: int(re.search(r'(\d+) basement\(s\)', x).group(1)) if isinstance(x, str) and re.search(r'(\d+) basement\(s\)', x) else 0)

df['Bedrooms'] = bedrooms
df['Bathrooms'] = bathrooms
df['Basements'] = basements

df = df.drop(columns=['Rooms'])

In [77]:
df

,Price,Type,Status,Parking Spaces,Land Size,Building Size,Architecture Style,County,Provider,Bedrooms,Bathrooms,Basements
0,2200.0,House,Active,NaN,NaN,136.20,Other,Mohave,"Black Mountain Valley Realty, Inc",3,2,0
1,3000.0,Apartment,Active,NaN,161.87,205.13,Other,PHILADELPHIA,Compass,2,3,0
2,1195.0,Apartment,Active,NaN,768.90,NaN,Other,PHILADELPHIA,OCF Realty,1,1,0
3,8000.0,Townhouse,Active,NaN,728.43,213.68,Other,Ventura,eXp Realty of California Inc.,4,3,0
4,5600.0,House,Active,4.0,849.84,266.72,Colonial,FAIRFAX,Keller Williams Fairfax Gateway,4,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3995,2050.0,Other,Active,NaN,566.56,175.03,Contemporary,Maricopa,Realty ONE Group,3,2,0
3996,1300.0,Other,Active,NaN,930.78,104.98,Ranch,Maricopa,Real Living SJ Fowler Real Estate,2,1,0
3997,3499.0,Other,Active,NaN,890.31,345.79,Contemporary,Maricopa,HomeSmart,4,3,0
3998,10900.0,Other,Active,NaN,1254.53,653.94,Other,Maricopa,Russ Lyon Sotheby's International Realty,5,7,0


### 1.1. Handle Missing Data

### 1.2. Handle Duplicate Rows

### 1.3. Other cleaning methods:

## 2. Features selection

## 3. Data normalization

# II. Exploration

- What is the meaning of each column?

- What is the current data type of each column? Are there columns having inappropriate data types?

- With each column, how are values distributed?